# Combined General Market Activity

This notebook makes tables 3, 4, and 5.

In [1]:
import os
import sys
import re

from itertools import *
from operator import mul
from functools import reduce
import pickle

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

idx = pd.IndexSlice

from research_tools import storage

pd.options.display.float_format = lambda x: '{:,.4f}'.format(x) if abs(x) < 1 else '{:,.2f}'.format(x)

In [2]:
os.chdir('..')

basename = 'dem'

dem_orders, dem_behavior_analysis, dem_trader_analysis, dem_quotes_dict = storage.retrieve_all(
    [basename + '.orders',
     basename + '.behavior_analysis',
     basename + '.trader_analysis',
     basename + '.reconstructed_quotes'])

basename = 'gop'

rep_orders, rep_behavior_analysis, rep_trader_analysis, rep_quotes_dict = storage.retrieve_all(
    [basename + '.orders',
     basename + '.behavior_analysis',
     basename + '.trader_analysis',
     basename + '.reconstructed_quotes'])

Reading data from data/dem.orders.p
Reading data from data/dem.behavior_analysis.p
Reading data from data/dem.trader_analysis.p
Reading data from data/dem.reconstructed_quotes.p
Reading data from data/gop.orders.p
Reading data from data/gop.behavior_analysis.p
Reading data from data/gop.trader_analysis.p
Reading data from data/gop.reconstructed_quotes.p


In [3]:
combined_behavior_analysis = dem_behavior_analysis.add(rep_behavior_analysis, fill_value=0)
combined_trader_analysis = pd.concat([rep_trader_analysis, dem_trader_analysis])

# Total Market Activity

The following table is for Table 3.

In [4]:
market_activity = pd.concat([dem_trader_analysis[['quantity', 'notional', 'fee']].sum(),
                             rep_trader_analysis[['quantity', 'notional', 'fee']].sum()], axis=1)

market_activity.columns = ['Democrat', 'Republican']

In [5]:
market_activity.T

,quantity,notional,fee
Democrat,"2,642,398.00","1,271,811.34","11,718.86"
Republican,"6,016,470.00","2,892,290.36","23,096.63"


In [6]:
print(market_activity.T.to_latex())

\begin{tabular}{lrrr}
\toprule
{} &     quantity &     notional &       fee \\
\midrule
Democrat   & 2,642,398.00 & 1,271,811.34 & 11,718.86 \\
Republican & 6,016,470.00 & 2,892,290.36 & 23,096.63 \\
\bottomrule
\end{tabular}



The next table is Table 4.

In [7]:
dem_trader_analysis['day_of_trade'] = dem_trader_analysis.date_executed.dt.date

dem_daily_activity = dem_trader_analysis.groupby('day_of_trade')[['quantity', 'notional']].sum()
dem_daily_activity.index = pd.to_datetime(dem_daily_activity.index)

rep_trader_analysis['day_of_trade'] = rep_trader_analysis.date_executed.dt.date

rep_daily_activity = rep_trader_analysis.groupby('day_of_trade')[['quantity', 'notional']].sum()
rep_daily_activity.index = pd.to_datetime(rep_daily_activity.index)

In [8]:
percentile_activity = pd.concat([dem_daily_activity['quantity'],
                                 (dem_daily_activity.cumsum() / dem_daily_activity.sum(axis=0))['quantity'],
                                 rep_daily_activity['quantity'],
                                 (rep_daily_activity.cumsum() / rep_daily_activity.sum(axis=0))['quantity'],
                                 ],
                                axis=1)
percentile_activity.columns = pd.MultiIndex.from_tuples(
    [('Democrat', 'Volume'),
     ('Democrat', 'Cum. Volume (%)'),
     ('Republican', 'Volume'),
     ('Republican', 'Cum. Volume (%)')
    ])
percentile_activity.index.name = None

percentile_activity[('Democrat', 'Cum. Volume (%)')] *= 100
percentile_activity[('Republican', 'Cum. Volume (%)')] *= 100

out = percentile_activity[percentile_activity.index.isin(
    ['2015-07-01', '2015-09-01',
     '2016-01-01', '2016-01-13', '2016-01-20',
     '2016-01-27', '2016-01-28', '2016-01-29', '2016-01-30', '2016-01-31', '2016-02-01', '2016-02-02'])].copy()

out[('Democrat', 'Volume')] = out[('Democrat', 'Volume')].astype(int)
out[('Republican', 'Volume')] = out[('Republican', 'Volume')].astype(int)

out

Democrat                 Republican                
             Volume Cum. Volume (%)     Volume Cum. Volume (%)
2015-07-01      736          0.1711        430          0.1914
2015-09-01      512            2.32       1894          0.9952
2016-01-01     8312           12.75      20496           23.76
2016-01-13    14494           18.00      62560           31.06
2016-01-20    21626           23.97      48224           36.37
2016-01-27    44000           35.55      92142           46.24
2016-01-28    33504           36.82     142114           48.60
2016-01-29    66666           39.34     200356           51.93
2016-01-30    75296           42.19     278388           56.56
2016-01-31   117420           46.64     282128           61.25
2016-02-01   392516           61.49     753354           73.77
2016-02-02  1017550          100.00    1578254          100.00

In [9]:
print(out.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & Democrat &                 & Republican &                 \\
{} &   Volume & Cum. Volume (\%) &     Volume & Cum. Volume (\%) \\
\midrule
2015-07-01 &      736 &          0.1711 &        430 &          0.1914 \\
2015-09-01 &      512 &            2.32 &       1894 &          0.9952 \\
2016-01-01 &     8312 &           12.75 &      20496 &           23.76 \\
2016-01-13 &    14494 &           18.00 &      62560 &           31.06 \\
2016-01-20 &    21626 &           23.97 &      48224 &           36.37 \\
2016-01-27 &    44000 &           35.55 &      92142 &           46.24 \\
2016-01-28 &    33504 &           36.82 &     142114 &           48.60 \\
2016-01-29 &    66666 &           39.34 &     200356 &           51.93 \\
2016-01-30 &    75296 &           42.19 &     278388 &           56.56 \\
2016-01-31 &   117420 &           46.64 &     282128 &           61.25 \\
2016-02-01 &   392516 &           61.49 &     753354 &           73.77 \\
2016-02-02 

# Trader Behavior Summary

The below table is for Table 5.

The Holding Times values are calculated separately, in the Miscellaneous Measurements notebooks.

In [10]:
dem_summary = Series()
rep_summary = Series()
combined_summary = Series()

dem_summary['Traders'] = len(dem_behavior_analysis.index)
rep_summary['Traders'] = len(rep_behavior_analysis.index)
combined_summary['Traders'] = len(combined_behavior_analysis.index)

dem_summary['Orders Sent'] = dem_orders[dem_orders.user_guid.isin(dem_behavior_analysis.index)].shape[0]
rep_summary['Orders Sent'] = rep_orders[rep_orders.user_guid.isin(rep_behavior_analysis.index)].shape[0]
combined_summary['Orders Sent'] = dem_summary['Orders Sent'] + rep_summary['Orders Sent']

dem_summary['Money Risked (cumulative exposure, $)'] = dem_behavior_analysis.max_in_pool.sum()
rep_summary['Money Risked (cumulative exposure, $)'] = rep_behavior_analysis.max_in_pool.sum()
combined_summary['Money Risked (cumulative exposure, $)'] = combined_behavior_analysis.max_in_pool.sum()

dem_summary['Average Money Risked (cumulative exposure, $)'] = dem_behavior_analysis.max_in_pool.mean()
rep_summary['Average Money Risked (cumulative exposure, $)'] = rep_behavior_analysis.max_in_pool.mean()
combined_summary['Average Money Risked (cumulative exposure, $)'] = combined_behavior_analysis.max_in_pool.mean()

dem_summary['Average (pre-fee) Profit ($)'] = 0
rep_summary['Average (pre-fee) Profit ($)'] = 0
combined_summary['Average (pre-fee) Profit ($)'] = 0

dem_summary['Average (net-fee) Profit ($)'] = dem_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().mean()
rep_summary['Average (net-fee) Profit ($)'] = rep_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().mean()
combined_summary['Average (net-fee) Profit ($)'] = combined_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().mean()

dem_summary['Min (net-fee) Profit ($)'] = dem_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().min()
rep_summary['Min (net-fee) Profit ($)'] = rep_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().min()
combined_summary['Min (net-fee) Profit ($)'] = combined_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().min()

dem_summary['Max (net-fee) Profit ($)'] = dem_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().max()
rep_summary['Max (net-fee) Profit ($)'] = rep_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().max()
combined_summary['Max (net-fee) Profit ($)'] = combined_trader_analysis.groupby('user_guid')['pnl_net_fee'].sum().max()

summary = pd.concat([dem_summary, rep_summary, combined_summary], axis=1)

summary.columns = ['Democrat', 'Republican', 'Combined']

summary

,Democrat,Republican,Combined
Traders,"3,750.00","4,452.00","6,160.00"
Orders Sent,"33,959.00","62,551.00","96,510.00"
"Money Risked (cumulative exposure, $)","417,251.70","544,205.11","961,456.80"
"Average Money Risked (cumulative exposure, $)",111.27,122.24,156.08
Average (pre-fee) Profit ($),0.0000,0.0000,0.0000
Average (net-fee) Profit ($),-3.13,-5.19,-5.65
Min (net-fee) Profit ($),"-1,699.58","-1,701.32","-2,429.86"
Max (net-fee) Profit ($),"1,187.54","2,882.00","3,101.20"


In [11]:
print(summary.to_latex())

\begin{tabular}{lrrr}
\toprule
{} &   Democrat &  Republican &   Combined \\
\midrule
Traders                                       &   3,750.00 &    4,452.00 &   6,160.00 \\
Orders Sent                                   &  33,959.00 &   62,551.00 &  96,510.00 \\
Money Risked (cumulative exposure, \$)         & 417,251.70 &  544,205.11 & 961,456.80 \\
Average Money Risked (cumulative exposure, \$) &     111.27 &      122.24 &     156.08 \\
Average (pre-fee) Profit (\$)                  &     0.0000 &      0.0000 &     0.0000 \\
Average (net-fee) Profit (\$)                  &      -3.13 &       -5.19 &      -5.65 \\
Min (net-fee) Profit (\$)                      &  -1,699.58 &   -1,701.32 &  -2,429.86 \\
Max (net-fee) Profit (\$)                      &   1,187.54 &    2,882.00 &   3,101.20 \\
\bottomrule
\end{tabular}

